In [ ]:
import os
import re
from pathlib import Path

import numpy as np
from PIL import Image

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
from violet.utils.preprocessing import extract_and_write_multichannel
from violet.utils.dataloaders import listfiles
from violet.utils.multichannel import create_pseudocolor_image, create_pseudocolor_image_from_dict

In [ ]:
from mgitools.os_helpers import listfiles

###### linking

In [ ]:
fps = sorted(listfiles('/home/estorrs/imaging/data/codex/codex_processed_hu_pancreas_TrisEDTA/codex_processed/codex_processed_hu_pancreas_TrisEDTA_2020-10-08/stitched/', regex=r'.tif$'))
fps

In [ ]:
d = '../data/codex/test/hu_pancreas_TrisEDTA/'

In [ ]:
# temporarily remap
import subprocess
for fp in fps:
    identifier = fp.split('/')[-1]
    identifier = re.sub(r'^.*_.*_.*_(.*).tif$', r'\1', identifier)
    new = os.path.join(d, f'{identifier}.tif')
    c = f'ln -s {fp} {new}'
    subprocess.check_output(c, shell=True)

#### bodenmiller

In [ ]:
fps = sorted(listfiles('/data/multiplex/bodenmiller_2019/extracted/tifs/', regex=r'.tif$'))
fps

In [ ]:
channels = sorted(set([f.split('/')[-1].replace('.tif', '') for f in fps]))
channels

In [ ]:
remove = ['NaN', 'nan', 'undefined']
fps = [fp for fp in fps if fp.split('/')[-1].replace('.tif', '') not in remove]

In [ ]:
data_map = {}
for fp in fps:
    sample = fp.split('/')[-2]
    if sample not in data_map:
        data_map[sample] = {}

    channel = fp.split('/')[-1].replace('.tif', '')
    data_map[sample][channel] = fp
len(data_map)

In [ ]:
Counter([c for s, d in data_map.items() for c in d.keys()]).most_common()

In [ ]:
data_map.keys()

In [ ]:
for k, channel_dict in data_map.items():
    print(k)
    pseudo = create_pseudocolor_image_from_dict(channel_dict, ['Histone H3', 'CD20', 'Vimentin', 'pan Cytokeratin'])
    plt.imshow(pseudo)
    plt.show()

In [ ]:
# !rm -rf /home/estorrs/violet/data/imc/bodenmiller_2019_128res

In [ ]:
val_samples = ['BaselTMA_SP41_15.475kx12.665ky_10000x8500_5_20170905_115_114_X14Y4_225_a0_full',
              'BaselTMA_SP41_33.475kx12.66ky_8500x8500_2_20170905_34_45_X4Y3_173_a0_full',
              'ZTMA208_slide_13.25kx21.95ky_7000x7000_9_20171115_363_28_Cy14x3_376_a0_full']

In [ ]:
output_dir = '/home/estorrs/violet/data/imc/bodenmiller_2019_128res'
extract_and_write_multichannel(data_map, output_dir, resolution=128., background_thres=None, background_channel=None,
                              val_samples=val_samples)

In [ ]:
import tifffile

In [ ]:
img = tifffile.imread('/data/multiplex/bodenmiller_2019/extracted/tifs/BaselTMA_SP41_15.475kx12.665ky_10000x8500_5_20170905_109_83_X13Y2_116_a0_full/Histone H3.tif')
img

In [ ]:
img.shape

In [ ]:
np.max(img)

In [ ]:
np.sum(img)

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img / np.max(img))

In [ ]:
for fp in fps:
    if 'EpCAM' in fp:
        img = tifffile.imread(fp)
        img[img>10.] = 10.
        plt.imshow(img)
        plt.show()

In [ ]:
for c, fp in data_map['BaselTMA_SP41_15.475kx12.665ky_10000x8500_5_20170905_124_31_X15Y2_140_a0_full'].items():
    print(c)
    plt.imshow(tifffile.imread(fp))
    plt.show()

#### multidataset integration

In [ ]:
dirs = ['/data/multiplex/codex_inhouse', '/data/multiplex/codex_public',
       '/data/multiplex/bodenmiller_2019/extracted/tifs/', '/data/multiplex/mcmicro/sardana/']

In [ ]:
# gather files
fps = []
for d in dirs:
    fps += [(d, fp) for fp in sorted(listfiles(d, regex=r'.tif$'))]
fps

In [ ]:
def clean_channel(c):
    if 'reg' in c and 'ch00' in c:
        return re.sub(r'^reg.*ch[0-9]{3}_(.*)$', r'\1', c)
    return c

In [ ]:
data_map = {}
for (dataset, fp) in fps:
    if dataset not in data_map:
        data_map[dataset] = {}
    sample = fp.split('/')[-2]
    if sample not in data_map[dataset]:
        data_map[dataset][sample] = {}

    channel = fp.split('/')[-1].replace('.tif', '')
    channel = clean_channel(channel)
    data_map[dataset][sample][channel] = fp
len(data_map)

In [ ]:
for dataset, m in data_map.items():
    print(dataset, Counter([c for s, d in m.items() for c in d.keys()]).most_common())
#     print(dataset, set([c for s, d in m.items() for c in d.keys()]))

In [ ]:
# see all channels
sorted(set([c for s, d in data_map.items() for c in d.keys()]))

In [ ]:
# convert dapi
for s, d in data_map.items():
    rem = [k for k in d.keys() if 'DAPI' in k]
    d['DAPI'] = str(d['DAPI1'])
    for k in rem:
        d.pop(k)

In [ ]:
from collections import Counter
ks = [k for s, d in data_map.items() for k in d]
Counter(ks).most_common()

In [ ]:
sorted(set(ks))

In [ ]:
output_dir = '/home/estorrs/violet/data/codex/tile_test_panc_07062021_512res_no_backround'
extract_and_write_multichannel(data_map, output_dir, resolution=512., background_thres=200.)

#### codex in-house

In [ ]:
d = '/data/multiplex/codex_inhouse/'

In [ ]:
fps = sorted(listfiles(d, regex=r'.tif$'))
fps

In [ ]:
def clean_channel(c):
    if 'reg' in c and 'ch00' in c:
        return re.sub(r'^reg.*ch[0-9]{3}_(.*)$', r'\1', c)
    return c

In [ ]:
data_map = {}
for fp in fps:
    sample = fp.split('/')[-2]
    if sample not in data_map:
        data_map[sample] = {}
        
    channel = fp.split('/')[-1].replace('.tif', '')
    channel = clean_channel(channel)
    data_map[sample][channel] = fp
len(data_map), data_map.keys()

In [ ]:
# convert dapi
for k, d in data_map.items():
    rem = [k for k in d.keys() if 'DAPI' in k]
    d['DAPI'] = str(d['DAPI1'])
    for k in rem:
        d.pop(k)

In [ ]:
from collections import Counter
ks = [k for s, d in data_map.items() for k in d]
Counter(ks).most_common()

In [ ]:
output_dir = '/home/estorrs/violet/data/codex/pdac_codex_256res'
extract_and_write_multichannel(data_map, output_dir, resolution=256., background_thres=100.)